In [3]:
import requests
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# News API endpoint and parameters
api_key = '33e4d796e2b44c3eb2f547afd088edf4'
base_url = 'https://newsapi.org/v2/everything'

def fetch_news(stock_name, from_date, to_date, language='en', page_size=100):
    params = {
        'q': stock_name,
        'from': from_date,
        'to': to_date,
        'language': language,
        'sortBy': 'relevancy',
        'pageSize': page_size,
        'apiKey': api_key
    }
    response = requests.get(base_url, params=params)
    articles = response.json().get('articles', [])
    
    data = []
    for article in articles:
        # Extract relevant information
        title = article['title']
        description = article['description']
        published_date = article['publishedAt'][:10]  # Format YYYY-MM-DD
        content = f"{title} {description}" if description else title
        # Calculate sentiment score
        sentiment = analyzer.polarity_scores(content)['compound']
        data.append({'Date': published_date, 'Content': content, 'Sentiment': sentiment})
    
    return pd.DataFrame(data)

# Example usage
end_date = datetime.now()
start_date = end_date - timedelta(days=7)
news_df = fetch_news('Suzlon', start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
news_df.head()
news_df.to_csv('newsapi.csv')